In [1]:
import transformers
from transformers import pipeline
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the model and tokenizer
def load_model(model_name="mosaicml/mpt-7b-instruct"):
    # Load the configuration with trust_remote_code set to True
    config = transformers.AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    model = transformers.AutoModelForCausalLM.from_pretrained(model_name, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
    model.to(device)
    return model, tokenizer

# Setup the pipeline
def setup_pipeline(model, tokenizer):
    pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)
    return pipe

# Function to generate text based on prompts
# Function to generate text based on prompts
def generate_text(prompt, pipe, max_tokens=100):
    response = pipe(
        prompt, 
        max_length=max_tokens, 
        num_return_sequences=1
    )
    return response[0]['generated_text']


Using device: cuda


In [3]:
# Load model and tokenizer
model_name = "mosaicml/mpt-7b-instruct"  # You can change this to your specific model
model, tokenizer = load_model(model_name)
pipe = setup_pipeline(model, tokenizer)

/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# Import necessary libraries
import os

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

# Define the function to read the prompt from a file
def read_prompt_from_file(file_path):
    with open(file_path, 'r') as file:
        prompt = file.read().strip()
    return prompt

# Define the path to the prompt file
prompt_file_path = 'prompt.txt'

# Read the prompt from the file
instruction = read_prompt_from_file(prompt_file_path)

# Format the prompt according to the specified format
formatted_prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

# Print the formatted prompt to verify
print("Formatted Prompt:\n", formatted_prompt)

# Assuming 'generate_text' and 'pipe' are defined elsewhere in your code
output = generate_text(formatted_prompt, pipe, max_tokens=2000)  # Increase max_tokens if needed

# Print the generated text
print("Generated Text:\n", output)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Formatted Prompt:
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
The potential fallacious argument types are:
- slippery slope
- ad hominem
- appeal to (false) authority
- X appeal to majority
- No fallacy

Which one of these 5 fallacious argument types does the following text contain?
"Man, that slope was slippery as hell."

Before identifying the fallacy, explain your reasoning thoroughly. Your explanation should clarify why the specific fallacy applies to the given statement. This step is crucial!
### Response:

Generated Text:
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
The potential fallacious argument types are:
- slippery slope
- ad hominem
- appeal to (false) authority
- X appeal to majority
- No fallacy

Which one of these 5 fallacious argument types does the following text contain?
"Man, that slope was slippery as hell."
